In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals = number of currencies
#### M = number of words only used by each topic
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, M, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    currencies = ['currency_' + str(i) for i in range(K)]
    random_capitals = ['random_capital_' + str(i) for i in range(M)]
    random_currencies = ['random_currency_' + str(i) for i in range(M)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + currencies + random_capitals + random_currencies + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []
        
        temp = npr.uniform()
        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        if temp <= 0.5: ### country - capital
        
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
#             pairs2 = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.8:
#                     sentence.append(random_capitals[pair])
#                 else:
#                     sentence.append(random_currencies[pair])

            randoms_dup = 4 * random_capitals + 2 * randoms + 1 * random_currencies
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
                 
        else: ### country - currency
            
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
            pairs2 = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(currencies[pair])        
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.8:
#                     sentence.append(random_currencies[pair])
#                 else:
#                     sentence.append(random_capitals[pair])
            
            randoms_dup = 1 * random_capitals + 2 * randoms + 4 * random_currencies
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
            
#         sentence += list(np.random.choice(randoms, L - 3 * n_pairs, replace = False))  

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [8]:
def get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, M, S, L, q1, q2, embed_dim, n_sentences)

    acc_capitals = []
    prob_capitals = []

    for _ in range(n_samples):
        sentence = []
        random_capitals = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_capital in random_capitals:
            sentence.append('country_' + str(random_capital))
            sentence.append('capital_' + str(random_capital))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_capitals[-1])]
        acc_capitals.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_capitals.append(temp[0][vocab_map['capital_' + str(random_capitals[-1])]])
        
    acc_currencies = []
    prob_currencies = []

    for _ in range(n_samples):
        sentence = []
        random_currencies = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_currency in random_currencies:
            sentence.append('country_' + str(random_currency))
            sentence.append('currency_' + str(random_currency))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['currency_' + str(random_currencies[-1])]
        acc_currencies.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_currencies.append(temp[0][vocab_map['currency_' + str(random_currencies[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_capitals), np.mean(prob_capitals)), \
                (np.mean(acc_currencies), np.mean(prob_currencies))

In [9]:
K = 10 # number of countries
L = 8 # sentence length
M = 20 # number of words used by each topic
S = 20 # number of words used by both topics
embed_dim = 10 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [22]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 8.3214595e-17)
(0.0, 1.4009464e-16)
(0.0, 1.462113e-14)
(0.0, 8.3089683e-14)
(0.0, 1.1292068e-14)
(0.0, 3.2792918e-14)
(0.0, 1.9938461e-14)
(0.0, 1.457133e-18)
(0.0, 8.692915e-18)
(0.0, 1.3593242e-17)
(0.0, 1.6518049e-12)
(0.0, 2.6100677e-13)
(0.0, 1.5436694e-13)
(0.0, 1.02521006e-10)
(0.0, 9.3441714e-14)
(0.0, 1.4293558e-13)
(0.0, 3.9450636e-17)
(0.0, 1.250635e-15)
(0.0, 5.0616144e-15)
(0.0, 2.0085748e-16)
(0.0, 1.950658175454245e-13)
(0.0, 1.0304243715442546e-11)


In [23]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 3.0484634e-07)
(0.0, 2.4450063e-07)
(0.0, 1.8634066e-05)
(0.0, 2.783521e-05)
(0.0, 1.7929342e-13)
(0.0, 7.7827485e-13)
(0.0, 3.0078863e-07)
(0.0, 4.5519147e-07)
(0.0, 3.52213e-06)
(0.0, 1.1548773e-06)
(0.0, 4.26297e-06)
(0.0, 2.9786926e-07)
(0.0, 2.4727813e-06)
(0.0, 6.087891e-06)
(0.0, 0.0004737468)
(0.0, 0.00031728655)
(0.0, 3.5114796e-09)
(0.0, 2.798786e-07)
(0.0, 2.5706981e-07)
(0.0, 6.995911e-08)
(0.0, 5.035049605769317e-05)
(0.0, 3.537119335315171e-05)


In [24]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.72, 0.502199)
(0.694, 0.46673694)
(0.825, 0.70494384)
(0.736, 0.60295236)
(0.573, 0.4654318)
(0.477, 0.3770471)
(0.786, 0.6514751)
(0.723, 0.63462037)
(0.81, 0.68875927)
(0.727, 0.5743369)
(0.778, 0.60464907)
(0.741, 0.5742794)
(0.689, 0.5197191)
(0.776, 0.68141174)
(0.512, 0.38633996)
(0.599, 0.46602094)
(0.786, 0.5728896)
(0.864, 0.71857214)
(0.373, 0.27059686)
(0.507, 0.3280563)
(0.6852, 0.5367003619670868)
(0.6844, 0.5424034208059312)


In [10]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

2024-05-01 13:02:17.755421: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-01 13:02:17.755453: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-01 13:02:17.755470: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl3471.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2024-05-01 13:02:17.755730: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(1.0, 0.99926966)
(1.0, 0.9982506)
(1.0, 0.9990663)
(1.0, 0.99929607)
(1.0, 0.99965537)
(1.0, 0.9978787)
(1.0, 0.9984031)
(1.0, 0.99948514)
(1.0, 0.99854124)
(1.0, 0.9973472)
(1.0, 0.99922097)
(1.0, 0.9976323)
(1.0, 0.9997227)
(1.0, 0.9988722)
(1.0, 0.9995129)
(1.0, 0.998846)
(1.0, 0.9996017)
(1.0, 0.9989794)
(1.0, 0.99902964)
(1.0, 0.9989065)
(0.9999999999999999, 0.9992023587226869)
(0.9999999999999999, 0.9985494077205659)


In [11]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.98765427)
(1.0, 0.9946565)
(1.0, 0.99299246)
(1.0, 0.9943016)
(1.0, 0.98487014)
(1.0, 0.9926157)
(1.0, 0.9971343)
(1.0, 0.99226826)
(1.0, 0.99350095)
(1.0, 0.9850159)
(1.0, 0.99728906)
(1.0, 0.9936302)
(1.0, 0.99172324)
(1.0, 0.9945902)
(1.0, 0.991106)
(1.0, 0.99587023)
(1.0, 0.98808455)
(1.0, 0.99075085)
(1.0, 0.9936555)
(1.0, 0.9970682)
(0.9999999999999999, 0.9918010473251343)
(0.9999999999999999, 0.9930767714977264)


In [12]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9966531)
(1.0, 0.9970186)
(1.0, 0.99799836)
(1.0, 0.9940305)
(1.0, 0.99430984)
(1.0, 0.9984759)
(1.0, 0.9956707)
(1.0, 0.9973306)
(1.0, 0.9982028)
(1.0, 0.99786687)
(1.0, 0.99744016)
(1.0, 0.9959179)
(1.0, 0.99782306)
(1.0, 0.9973954)
(1.0, 0.9984711)
(1.0, 0.998881)
(1.0, 0.99778235)
(1.0, 0.997685)
(1.0, 0.9978583)
(1.0, 0.9979493)
(0.9999999999999999, 0.9972209692001344)
(0.9999999999999999, 0.9972551047801971)
